In [1]:
using Mmap
using Match
using BenchmarkTools
using Profile
using Distributed
Threads.nthreads()


6

In [2]:
function readtoend()
    res = UInt64(0)
    f = open("./COX3.txt", "r")
    line = ""
    while !eof(f)
        line = readline(f)
#         res += sum(line)
    end
    line
end

function mmap_readtoend(step::Int)
    res = UInt64(0)
    f = open("./COX3.txt", "r")
    m = Mmap.mmap(f)
    block = zeros(UInt8, step)
    s = 1
    e = 1
    l = length(m)
    while e < l
        e += step
        if e > l
            e = l
        end
        
        res += sum(m[s: e])
        s += step
        
    end
    res
end

@time readtoend()
@time mmap_readtoend(65536)

  2.982507 seconds (27.10 M allocations: 828.550 MiB, 2.37% gc time)
  0.153480 seconds (12.38 k allocations: 387.347 MiB, 10.31% gc time)


0x00000003dd0aaeed

In [29]:
function get_mmap()
#     f = open("./COX3.txt", "r")
    f = open("/home/myuan/projects/data/T33-all.txt", "r")
    m = Mmap.mmap(f)
    m
end

function addx(x::UInt, n::UInt8)
    return (x * 10 + n)::UInt
end

function mmap_read_limit(start_pos::Int, end_pos::Int)

    m = get_mmap()
    current_pos = start_pos
    current_status = 1

    l = length(m)
    if end_pos === 0
        end_pos = l
    end

    current_x = UInt(0)
    current_y = UInt(0)
    max_x = max_y = UInt(0)
    min_x = min_y = typemax(UInt)

    while current_pos < end_pos && current_pos < l
        @inbounds current_char = m[current_pos]
        if current_char === UInt8('\t')
            current_status += 1 
        elseif current_char === UInt8('\n')
            current_status = 1
            max_x = max(max_x, current_x)
            max_y = max(max_y, current_y)
            min_x = min(min_x, current_x)
            min_y = min(min_y, current_y)
            current_x = current_y = UInt(0)
    #         println("$max_x, $max_y, $min_x, $min_y\n")
        else
            if current_status === 1 
                current_x = addx(current_x, current_char - UInt8('0'))::UInt
            elseif current_status === 2
                current_y = addx(current_y, current_char - UInt8('0'))::UInt
            end

        end


        current_pos += 1
    #     print("$current_pos, $current_status, $current_x, $current_y, $(Char(current_char))\n")
    #     if current_pos > 30
    #         break
    #     end
    end
#     print("$current_pos, $current_status, $current_x, $current_y, $(Char(current_char))\n")
    current_pos, max_x, max_y, min_x, min_y
end

mmap_read_limit (generic function with 1 method)

In [ ]:
@benchmark mmap_read_limit(0, 0)

In [34]:
@time mmap_read_limit(1, 0)

 40.871332 seconds (17 allocations: 1.062 KiB)


(22567501017, 0x001a3dec0b47de7a, 0x000000000000ef42, 0x0000000000000024, 0x0000000000000000)

In [7]:
@code_warntype mmap_read_limit(1, 0)

Variables
  #self#::Core.Const(mmap_read_limit)
  start_pos::Int64
  end_pos@_3::Int64
  min_x::UInt64
  min_y::UInt64
  max_x::UInt64
  max_y::UInt64
  current_y::UInt64
  current_x::UInt64
  l::Int64
  current_status::Int64
  current_pos::Int64
  m::Vector{UInt8}
  val::UInt8
  current_char::UInt8
  end_pos@_16::Int64
  @_17::Bool

Body::Tuple{Int64, UInt64, UInt64, UInt64, UInt64}
1 ──       (end_pos@_16 = end_pos@_3)
│          Core.NewvarNode(:(min_x))
│          Core.NewvarNode(:(min_y))
│          Core.NewvarNode(:(max_x))
│          Core.NewvarNode(:(max_y))
│          Core.NewvarNode(:(current_y))
│          Core.NewvarNode(:(current_x))
│          (m = Main.get_mmap())
│          (current_pos = start_pos)
│          (current_status = 1)
│          (l = Main.length(m))
│    %12 = (end_pos@_16 === 0)::Bool
└───       goto #3 if not %12
2 ──       (end_pos@_16 = l)
3 ┄─       (current_x = Main.UInt(0))
│          (current_y = Main.UInt(0))
│    %17 = Main.UInt(0)::Core.Const(0x0

## 并行版本

In [8]:
function find_line_end_helper(m::Vector{UInt8}, start::Int)
    
    while start <= length(m)
        if m[start] == UInt8('\n')
            break
        else
            start += 1
        end
    end
    
    return start
end


function find_line_end(m::Vector{UInt8}, chunk_size::Int)
    l = length(m)
    positions = Tuple{Int, Int}[]
    start = 1
    
    single_size = floor(Int, l / chunk_size)
    
    for i = 1:chunk_size
        new_start = start + single_size
        if new_start > l
            new_start = l
        end
        
        new_start = find_line_end_helper(m, new_start)
        
        push!(positions, (start, new_start))
        start = new_start + 1
    end
    positions
end
    

find_line_end (generic function with 1 method)

In [15]:
@time find_line_end(get_mmap(), 1)
@code_warntype find_line_end(get_mmap(), 1)

  0.000102 seconds (19 allocations: 1.219 KiB)
Variables
  #self#::Core.Const(find_line_end)
  m::Vector{UInt8}
  chunk_size::Int64
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  single_size::Int64
  start::Int64
  positions::Vector{Tuple{Int64, Int64}}
  l::Int64
  i::Int64
  new_start::Int64

Body::Vector{Tuple{Int64, Int64}}
1 ─       (l = Main.length(m))
│   %2  = Core.apply_type(Main.Tuple, Main.Int, Main.Int)::Core.Const(Tuple{Int64, Int64})
│         (positions = Base.getindex(%2))
│         (start = 1)
│   %5  = (l / chunk_size)::Float64
│         (single_size = Main.floor(Main.Int, %5))
│   %7  = (1:chunk_size)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_4 = Base.iterate(%7))
│   %9  = (@_4 === nothing)::Bool
│   %10 = Base.not_int(%9)::Bool
└──       goto #6 if not %10
2 ┄ %12 = @_4::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (i = Core.getfield(%12, 1))
│   %14 = Core.getfield(%12, 2)::Int64
│         (new_start = start + single_size)
│

In [35]:
function mmap_read_limit_pmap(size::Int)
    m = get_mmap()
    line_ends = find_line_end(m, size)
    pmap(i -> mmap_read_limit(i[1], i[2]), line_ends)
end
@time mmap_read_limit_pmap(6)

 42.203565 seconds (78.33 k allocations: 4.529 MiB, 0.17% compilation time)


6-element Vector{Tuple{Int64, UInt64, UInt64, UInt64, UInt64}}:
 (3761250170, 0x001a3dec0b47de7a, 0x000000000000ef42, 0x0000000000000024, 0x0000000000000000)
 (7522500355, 0x001a3dec0b47de7a, 0x000000000000ef42, 0x0000000000000024, 0x0000000000000000)
 (11283750528, 0x001a3dec0b47de7a, 0x000000000000ef42, 0x0000000000000024, 0x0000000000000000)
 (15045000713, 0x001a3dec0b47de7a, 0x000000000000ef42, 0x0000000000000024, 0x0000000000000000)
 (18806250898, 0x001a3dec0b47de7a, 0x000000000000ef42, 0x0000000000000024, 0x0000000000000000)
 (22567501017, 0x001a3dec0b47de7a, 0x000000000000ef42, 0x0000000000000024, 0x0000000000000000)

In [36]:
arr = rand(UInt8, 3, 50, 50)
img = colorview(RGB, arr / 255)


LoadError: UndefVarError: colorview not defined

In [40]:
using TestImages, Images, ImageView
img = testimage("mandrill")
imshow(img)


LoadError: ArgumentError: Package ImageView not found in current path:
- Run `import Pkg; Pkg.add("ImageView")` to install the ImageView package.
